In [ ]:
!pip install awswrangler

In [ ]:
from pathlib import Path
import pandas as pd
import awswrangler as wr

s3_output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/'
target_database = 'sandbox_measurement_analyst'
target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'
cur = Path('/root/MARS_ETL_PROCESSES/sql/actemra/etl_tables')
sql_files = sorted([file for file in cur.iterdir() if str(file).endswith('.sql')])

for sql_file in sql_files:
    table_name = sql_file.stem
    with open(sql_file, 'r') as file:
        sql_query = file.read()
        
    df = wr.athena.read_sql_query(
        sql=sql_query,
        database=target_database,
        workgroup=target_workgroup
    )
    
    s3_table_path = f"{s3_output_path}{table_name}/"
    
    wr.s3.to_parquet(
        df=df,
        path=s3_table_path,
        dataset=True,
        mode='overwrite'
    )
    print(f"Data for {table_name} written to {s3_table_path}")

print('done')